In [253]:
import pandas as pd
import re
import datetime
from gssutils import *
from rdflib import Graph, Literal, RDF, URIRef
import rdflib
from rdflib.namespace import OWL, VOID, DCTERMS, RDF, RDFS, SKOS, XSD, FOAF, DC
import json

In [254]:
are = pd.read_csv('ref_common/reference/codelists/cl-area.csv')
are['Notation'][are['Label'] == 'Namibia'] = 'NA' # Namibia Notation comes through as a NaN
are2 = are[are['Description'].notnull()]
are3 = are[are['Description'].isna()]
are3.head(5)

<ipython-input-254-b76a53e191f4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  are['Notation'][are['Label'] == 'Namibia'] = 'NA' # Namibia Notation comes through as a NaN


,Label,Notation,Parent Notation,Sort Priority,Description
0,Not allocated/ unspecified,_X,NaN,1,NaN
1,Not applicable,_Z,NaN,2,NaN
4,IMF (International Monetary Fund),1C,NaN,5,NaN
5,WTO (World Trade Organisation),1D,NaN,6,NaN
6,IBRD (International Bank for Reconstruction an...,1E,NaN,7,NaN


In [255]:
#### Persian Gulf Economies (CDIS),S35E,,718,S35E = BH + IQ + IR + KW + OM + QA + SA + AE + A35E9

In [256]:
def one_thing_equals_other_things(descr, lab):
    #### Example = 4AA=4B+4C+4D+4E+4G+4H+4I+4J+4R+4T+4U+4V+4S
    meth_dat = pd.DataFrame(columns=['Narrower', 'Main', 'Main Label'])
    s1 = descr.split('=', 1) # Split into left side of = sign and right
    s2 = s1[1].split('+')    # split the right side by +
    s1 = s1[0]
    for r in s2:
        meth_dat = pd.concat([meth_dat, pd.DataFrame({'Narrower':[r.strip()], 'Main':[s1.strip()], 'Main Label':[lab]})], sort=False)
    return meth_dat


def one_equals_others_plus_some_minus_some(descr, lab):
    #### Example = B5 = I8 + (BG + CZ + DK + GB + HR + HU + PL + RO + SE+ 4A + B09) -U29
    meth_dat = pd.DataFrame(columns=['Narrower', 'Main', 'Main Label'])
    s1 = descr.split('-', 1) 
    s1 = s1[0].split('=', 1)
    s1[1] = s1[1].replace('(','').replace(')','')
    s2 = s1[1].split('+')
    for r in s2:
        meth_dat = pd.concat([meth_dat, pd.DataFrame({'Narrower':[r.strip()], 'Main':[s1[0].strip()], 'Main Label':[lab]})], sort=False)
    return meth_dat


def one_equals_others_minus_some(descr, lab):
    #### D5A = D5 - (BR + CA + CN + IN + JP + RU + CH +US)
    #meth_dat = pd.DataFrame(columns=['Narrower', 'Main'])
    s1 = frame.split('=',1)
    meth_dat = pd.DataFrame({'Narrower':['None'], 'Main':[s1[0].strip()], 'Main Label':[lab]})
    return meth_dat


def one_equals_one_and_minus_another(descr, lab):
    #### Example = A3X=A3-MX
    s1 = frame.split('=',1)
    meth_dat = pd.DataFrame({'Narrower':['None'], 'Main':[s1[0].strip()], 'Main Label':[lab]})
    return meth_dat


def no_description(fram, lab, desc):   
    return pd.DataFrame({'Narrower':['None'], 'Main':[fram.strip()], 'Main Label':[lab.strip()]})


def all_pluses_in_description(descr, lab, fram):
    #### Example = AG + BB + BS + BZ + CU + DM + DO + GD + GY + HT + JM + KN + LC + SR + TT + VC
    meth_dat = pd.DataFrame(columns=['Narrower', 'Main', 'Main Label'])
    s1 = descr.split('+')
    for r in s1:
        r = r.replace('(','')
        meth_dat = pd.concat([meth_dat, pd.DataFrame({'Narrower':[r.strip()], 'Main':[fram.strip()], 'Main Label':[lab]})], sort=False)
    return meth_dat


def until_or_from_in_description(fram, lab, descr):
    return no_description(fram, lab, descr)


def all_minuses_in_description(fram, lab, descr):
    return no_description(fram, lab, descr)


def descr_has_from_until_upto(fram, lab, descr):
    now = datetime.datetime.now()
    yr = now.year
    addTo = False    
    unfro = pd.DataFrame(columns=['Narrower', 'Main', 'Main Label'])
    if '=' in desc:
        descr = descr.split('=',1)
        descr = descr[1]
    if '+' in descr:
        s1 = descr.split('+')

        for s in s1:
            if 'from' in s.lower():
                num = int((re.findall('\d+', s))[0])
                if num <= yr:
                    s = ((s.split('(',1))[0]).strip()
                    addTo = True
                else:
                    addTo = False
            elif 'until' in s.lower():
                num = int((re.findall('\d+', s))[0])
                if num >= yr:
                    s = ((s.split('(',1))[0]).strip()
                    addTo = True
                else:
                    addTo = False
            elif 'up to' in s.lower():
                num = int((re.findall('\d+', s))[0])
                if num >= yr:
                    s = ((s.split('(',1))[0]).strip()
                    addTo = True
                else:
                    addTo = False
            else:
                addTo = True
                
            if addTo:
                unfro = pd.concat([unfro, pd.DataFrame({'Narrower':[s.strip()],'Main':[fram.strip()],
                                'Main Label':[labe.strip()]})],sort=False)
    else:
        #print('Ignoring: ' + fram + ' === ' + desc)
        unfro = pd.concat([unfro, pd.DataFrame({'Narrower':[''],'Main':[fram.strip()],
                                'Main Label':[lab.strip()]})],sort=False)
        
    return unfro


def another_special_case(fram, lab, descr):
    # Descriptions with 'Country aggregate' or 'Candidate countries' in it
    agg = pd.DataFrame(columns=['Narrower', 'Main', 'Main Label'])
    s1 = descr.split(' (' , 1)
    s1[1] = s1[1].replace(')','')
    if '+' in s1[1]:
        s2 = s1[1].split('+')
    else:
        s2 = s1[1].split(',')
    for c in s2:
        agg = pd.concat([agg, pd.DataFrame({'Narrower':[c.strip()],'Main':[fram.strip()],
                        'Main Label':[lab.strip()]})],sort=False)
    
    return agg

def get_coordinates():
    with open('Dev Code/geo-countries_zip/archive/countries.geojson') as json_file:
        data = json.load(json_file)

    data = data["features"]

    co_dat = pd.DataFrame(columns=['Main Label', 'ISO', 'Shape', 'Coordinates'])
    for x in data:
        country = str(x['properties']['ADMIN']).strip()
        iso = str(x['properties']['ISO_A3']).strip()
        tpe = str(x['geometry']['type']).strip()
        coord = str(x['geometry']['coordinates'])
        coord = coord.replace('[','(')
        coord = coord.replace(']',')')
        co_dat = pd.concat([co_dat, pd.DataFrame({'Main Label':[country], 'ISO':[iso], 'Shape':[tpe], 'Coordinates':[coord]})], sort=False)
    
    return co_dat

In [257]:
dat1 = pd.DataFrame(columns=['Narrower', 'Main', 'Main Label'])
dat2 = pd.DataFrame(columns=['Narrower', 'Main', 'Main Label'])
dat3 = pd.DataFrame(columns=['Narrower', 'Main', 'Main Label'])
dat4 = pd.DataFrame(columns=['Narrower', 'Main', 'Main Label'])
dat5 = pd.DataFrame(columns=['Narrower', 'Main', 'Main Label'])
dat6 = pd.DataFrame(columns=['Narrower', 'Main', 'Main Label'])

for item, frame in are['Notation'].iteritems():
    desc = str((list(are['Description'][are['Notation'] == frame]))[0])
    labe = str((list(are['Label'][are['Notation'] == frame]))[0])
    desc = desc.strip().replace('  ', ' ')     
    if (('aggregate' in labe.lower()) or ('candidate' in labe.lower())) and (frame != 'C0'):
        desc = 'special case'
            
    if (frame != 'A2A39') and (desc != 'nan'):
        desc = desc.replace('W1=sum of all the (ISO 3166-1 alpha-2) partner countries +1A+W19;','')
        desc = desc.replace('sum of (ISO 3166-1 alpha-2) partner countries;','')
        desc = desc.replace('…','')
        desc = desc.replace('+…','')
        desc = desc.replace('(36 countries)','')
        desc = desc.replace('+ A35E9','') # Need to look into this as A35E9 is not in Notations
        desc = desc.replace('rule 1: ','')
        desc = desc.replace('+ T F+','+ TF +')
        if ('countries' in desc):
            for x in range(90):
                desc = desc.replace(f'({x} countries)','')  

        if 'special case' in desc:
            desc = labe
            new_dat = another_special_case(frame, labe, desc)
            dat6 = pd.concat([dat6,new_dat], sort=False)
        elif ('from' in desc.lower()) or ('until' in desc.lower()) or ('up to' in desc.lower()):
            new_dat = descr_has_from_until_upto(frame, labe, desc)
            dat6 = pd.concat([dat6,new_dat], sort=False)
            del new_dat
        elif ('=' in desc) and ('-' not in desc):
            new_dat = one_thing_equals_other_things(desc, labe)
            dat1 = pd.concat([dat1,new_dat], sort=False)
            del new_dat
        elif ('=' in desc) and ('-' in desc) and ('from' not in desc):
            if ('(' in desc) and (')' in desc):
                if ('=' in desc) and (('+ (' in desc) or ('+(' in desc)) and ('-' in desc):
                    new_dat = one_equals_others_plus_some_minus_some(desc, labe)
                    dat2 = pd.concat([dat2,new_dat], sort=False)
                    del new_dat
                elif ('=' in desc) and (('- (' in desc) or ('-(' in desc)) and ('+' in desc):
                    new_dat = one_equals_others_minus_some(desc, labe)
                    dat2 = pd.concat([dat2,new_dat], sort=False)
                    del new_dat
                else:
                    print(frame + ' -- ' + desc)
            elif ('=' in desc) and ('-' in desc):
                new_dat = one_equals_one_and_minus_another(desc, labe)
                dat2 = pd.concat([dat2,new_dat], sort=False)
                del new_dat
            else:
                print(frame + ' -- ' + desc)
        elif 'rule' in desc:
            new_dat = no_description(frame, labe, desc)
            dat3 = pd.concat([dat3,new_dat], sort=False)
            del new_dat
        elif ('-' not in desc) and ('(' not in desc):
            new_dat = all_pluses_in_description(desc, labe, frame)
            dat5 = pd.concat([dat5,new_dat], sort=False)
            del new_dat
        elif '+' in desc:
            new_dat = all_pluses_in_description(desc, labe, frame)
            dat5 = pd.concat([dat5,new_dat], sort=False)
            del new_dat
        else:
            new_dat = all_minuses_in_description(frame, labe, desc)
            dat3 = pd.concat([dat3,new_dat], sort=False)
            del new_dat
    else:
        new_dat = no_description(frame, labe, '')
        dat4 = pd.concat([dat4,new_dat], sort=False)
        del new_dat

In [258]:
print('Original number of codes: ' + str(len(are['Notation'].unique())))
print('dat1 count: ' + str(dat1['Main'].count()) + ' - Unique: ' + str(len(dat1['Main'].unique())))
print('dat2 count: ' + str(dat2['Main'].count()) + ' - Unique: ' + str(len(dat2['Main'].unique())))
print('dat3 count: ' + str(dat3['Main'].count()) + ' - Unique: ' + str(len(dat3['Main'].unique())))
print('dat4 count: ' + str(dat4['Main'].count()) + ' - Unique: ' + str(len(dat4['Main'].unique())))
print('dat5 count: ' + str(dat5['Main'].count()) + ' - Unique: ' + str(len(dat5['Main'].unique())))
print('dat6 count: ' + str(dat6['Main'].count()) + ' - Unique: ' + str(len(dat6['Main'].unique())))
dat1 = dat1.drop_duplicates()
dat2 = dat2.drop_duplicates()
dat3 = dat3.drop_duplicates()
dat4 = dat4.drop_duplicates()
dat5 = dat5.drop_duplicates()
dat6 = dat6.drop_duplicates()
maindat = pd.concat([dat1, dat2, dat3, dat4, dat5, dat6])
print('Before dropping duplicates from maindat: ' + str(len(maindat['Main'].unique())))
maindat = maindat.drop_duplicates()
print('How many Main codes have been processed: ' + str(len(maindat['Main'].unique())))
print('How many Narrower codes: ' + str(len(maindat['Narrower'].unique())))
# Merge on Main to get the description
maindat3 = pd.merge(maindat, are, how='left', left_on=['Main'], right_on=['Notation'])
maindat3 = maindat3.rename(columns={'Description':'Main Description'})
maindat3 = maindat3[['Narrower','Main','Main Label','Main Description']]
# Merge on Narrower to get the label and description
maindat3 = pd.merge(maindat3, are, how='left', left_on=['Narrower'], right_on=['Notation'])
maindat3 = maindat3.rename(columns={'Description':'Narrow Description', 'Label':'Narrow Label'})
maindat3 = maindat3[['Narrow Description','Narrow Label','Narrower','Main','Main Label','Main Description']]
print('=============================================================================')
print('Number of Main Codes after joining: ' + str(len(maindat3['Main'].unique())))
print('Number of Narrower codes after joining: ' + str(len(maindat3['Narrower'].unique())))
maindat = maindat3

Original number of codes: 879
dat1 count: 878 - Unique: 34
dat2 count: 25 - Unique: 14
dat3 count: 22 - Unique: 22
dat4 count: 691 - Unique: 691
dat5 count: 1571 - Unique: 90
dat6 count: 394 - Unique: 28
Before dropping duplicates from maindat: 879
How many Main codes have been processed: 879
How many Narrower codes: 363
Number of Main Codes after joining: 879
Number of Narrower codes after joining: 363


In [259]:
del dat1, dat2, dat3, dat4, dat5, dat6

In [260]:
# Set things to None rather than NaN cos i don't like NaNs
for c in maindat.columns:
    if (c != 'Main Description') and (c != 'Narrow Description'):
        maindat[c][maindat[c].isna()] = 'None'
        maindat[c][maindat[c] == ''] = 'None'
    
#maindat.to_csv("codeHierarchyhCheck.csv")

In [261]:
# Check to see if all codes have been captured
df = pd.merge(are, maindat, how='left', left_on='Notation', right_on='Main')
df = df[['Notation','Main']]
d = df[df['Main'].isna()]
# If nothing comes our of this then everything SHOULD be alright
d.head(60)

,Notation,Main


In [262]:
uniqueCodes = maindat['Main'].unique()
maindat.head(6)

,Narrow Description,Narrow Label,Narrower,Main,Main Label,Main Description
0,1B=1C+1D+1E+1F+1G+1H+1J+1K+1L+1M+1N+1O+1P+1Q+1...,UN organisations,1B,1A,International organisations,1A=1B+4A+4F+4S+5A+6A
1,4A=4B+4C+4D+4E+4G+4H+4I+4J+4M+4R+4T+4U+4V,All the European Union Institutions excluding ...,4A,1A,International organisations,1A=1B+4A+4F+4S+5A+6A
2,NaN,ECB (European Central Bank),4F,1A,International organisations,1A=1B+4A+4F+4S+5A+6A
3,NaN,ESM (European Stability Mechanism),4S,1A,International organisations,1A=1B+4A+4F+4S+5A+6A
4,5AA+5B+…+5Z+7A+…7M,Other International Organisations (financial i...,5A,1A,International organisations,1A=1B+4A+4F+4S+5A+6A
5,6B+…+6Z,Other International Organisations (non-financi...,6A,1A,International organisations,1A=1B+4A+4F+4S+5A+6A


In [263]:
# Get all values from the coordinate files
coords = get_coordinates()
coords.head(2)

,Main Label,ISO,Shape,Coordinates
0,Aruba,ABW,Polygon,"(((-69.99693762899992, 12.577582098000036), (-..."
0,Afghanistan,AFG,Polygon,"(((71.04980228700009, 38.40866445000009), (71...."


In [264]:
coords['Main Label'] = coords['Main Label'].str.strip()
# Got to rename some stuff to match up the two codelists
co = { "The Bahamas":"Bahamas",
    "Brunei":"Brunei Darussalam",
    "Democratic Republic of the Congo":"Congo, the Democratic Republic of the",
    "Republic of Congo":"Congo",
    "Curaçao":"Curacao",
    "Falkland Islands":"Falkland Islands (Malvinas)",
    "Federated States of Micronesia":"Micronesia, Federated States of",
    "Guinea Bissau":"Guinea-Bissau",
    "Hong Kong S.A.R.":"Hong Kong, China",
    "British Indian Ocean Territory":"British Indian Ocean territory",
    "Iran":"Iran, Islamic Republic of",
    "South Korea":"Korea, Republic of",
    "Laos":"Lao People`s Democratic Republic",
    "Macao S.A.R":"Macao",
    "Saint Martin":"Saint Martin (French part)",
    "Moldova":"Moldova, Republic of",
    "Macedonia":"Republic of North Macedonia",
    "Pitcairn Islands":"Pitcairn",
    "North Korea":"Korea, Democratic People`s Republic of",
    "Palestine":"Palestine, State of",
    "Russia":"Russian Federation",
    "South Georgia and South Sandwich Islands":"South Georgia and the South Sandwich Islands",
    "Republic of Serbia":"Serbia",
    "Sint Maarten":"Sint Maarten (Dutch part)",
    "Syria":"Syrian Arab Republic",
    "East Timor":"Timor-Leste",
    "Taiwan":"Taiwan, Province of China",
    "United Republic of Tanzania":"Tanzania, United Republic of",
    "United States Minor Outlying Islands":"United States Minor outlying islands",
    "United States of America":"United States",
    "Vatican":"Holy See (Vatican City State)",
    "Venezuela":"Venezuela, Bolivarian Republic",
    "British Virgin Islands":"Virgin Islands, British",
    "United States Virgin Islands":"Virgin Islands, U.S.",
    "Vietnam":"Viet Nam"
}

for key in co:
    coords['Main Label'][coords['Main Label'] == key] = co[key]
    
    

In [265]:
maindat = pd.merge(maindat,coords, on='Main Label', how='left')

maindat['ISO'][maindat['ISO'].isna()] = 'None'
maindat['ISO'][maindat['ISO'] == '-99'] = 'None'
maindat['Coordinates'][maindat['Coordinates'].isna()] = 'None'
maindat['Shape'][maindat['Shape'].isna()] = 'None'
    
maindat.head(6)

,Narrow Description,Narrow Label,Narrower,Main,Main Label,Main Description,ISO,Shape,Coordinates
0,1B=1C+1D+1E+1F+1G+1H+1J+1K+1L+1M+1N+1O+1P+1Q+1...,UN organisations,1B,1A,International organisations,1A=1B+4A+4F+4S+5A+6A,None,None,None
1,4A=4B+4C+4D+4E+4G+4H+4I+4J+4M+4R+4T+4U+4V,All the European Union Institutions excluding ...,4A,1A,International organisations,1A=1B+4A+4F+4S+5A+6A,None,None,None
2,NaN,ECB (European Central Bank),4F,1A,International organisations,1A=1B+4A+4F+4S+5A+6A,None,None,None
3,NaN,ESM (European Stability Mechanism),4S,1A,International organisations,1A=1B+4A+4F+4S+5A+6A,None,None,None
4,5AA+5B+…+5Z+7A+…7M,Other International Organisations (financial i...,5A,1A,International organisations,1A=1B+4A+4F+4S+5A+6A,None,None,None
5,6B+…+6Z,Other International Organisations (non-financi...,6A,1A,International organisations,1A=1B+4A+4F+4S+5A+6A,None,None,None


In [266]:
maindat[maindat['Main Label'] == "Macao"]
coords[coords['Main Label'] == "Macao S.A.R"]

,Main Label,ISO,Shape,Coordinates


In [268]:
# CHeck which country coordinates are missing, should be 21 left that are not in the main codelist
mainCnt = pd.DataFrame(list(maindat['ISO'].unique()))
mainCnt = mainCnt.rename(columns={0:"ISO"})

mainCnt = mainCnt.merge(maindat, on='ISO', how='left')
mainCnt = mainCnt[['ISO','Main Label']]
mainCnt = mainCnt[mainCnt['ISO'] != 'None']
mainCnt = list(mainCnt['Main Label'])

cordCnt = list(coords['Main Label'])

co_not = pd.DataFrame(columns=['Main Label'])
co_in = pd.DataFrame(columns=['Main Label'])

for c in cordCnt:
    if c not in mainCnt:     
        co_not = pd.concat([co_not, pd.DataFrame({'Main Label':[c]})], sort=False)
    else:
        co_in = pd.concat([co_in, pd.DataFrame({'Main Label':[c]})], sort=False)
        

#co_dat.to_csv('missing_countries.csv')
co_in.to_csv('countries_with_coords.csv', index=False)
pd.DataFrame(co_not).head(60)

,Main Label
0,Aland
0,Ashmore and Cartier Islands
0,French Southern and Antarctic Lands
0,Bajo Nuevo Bank (Petrel Is.)
0,Ivory Coast
0,Clipperton Island
0,Cyprus No Mans Area
0,Coral Sea Islands
0,Northern Cyprus
0,Dhekelia Sovereign Base Area


In [244]:
g = Graph()
repoBase = "http://gss-data.org.uk/def"
con = "concept"
sch = "scheme"
consch = con + "-" + sch
sortPri = 'http://www.w3.org/ns/ui#sortPriority'

mainLabel = "SDMX Area codelist"
main ="cl-area"
desc = "Dataset representing the SDMX Area codelist. URN: urn:sdmx:org.sdmx.infomodel.codelist.Codelist=IMF:CL_AREA(1.13). Main source: https://sdmx.org/?page_id=1747. Structure source: https://registry.sdmx.org/ws/public/sdmxapi/rest/datastructure/IMF/BOP/1.14/?detail=full&references=descendants&version=2.1:1:555:"

geomURI = URIRef('http://www.opengis.net/ont/geosparql#Geometry')
geomSpa = URIRef('http://www.opengis.net/ont/geosparql#asWKT')
hasGeom = URIRef('http://www.opengis.net/ont/geosparql#hasGeometry')

schemeURI = URIRef(repoBase + "/" + con + '-' + sch + "/" + main)
i = 1

datasetStr = '/dataset'
catrecStr = '/catalog-record'
# Set up the main schema triples
g.add((schemeURI, RDF.type, SKOS.ConceptScheme))
g.add((schemeURI, RDF.type, URIRef("http://publishmydata.com/pmdcat#ConceptScheme")))
g.add((schemeURI, URIRef("http://purl.org/dc/terms/title"), Literal(mainLabel, lang="en")))
g.add((schemeURI, RDFS.label, Literal(mainLabel, lang="en")))
g.add((schemeURI, RDFS.seeAlso, URIRef(schemeURI + datasetStr)))
g.add((schemeURI, URIRef("http://www.w3.org/ns/prov#hadDerivation"), URIRef(schemeURI)))
g.add((schemeURI, URIRef("http://gss-data.org.uk/catalog/vocabularies"), URIRef(schemeURI + catrecStr)))
 
datasetDateTime = "2021-03-04T11:40:13Z"
g.add((URIRef(schemeURI + datasetStr), RDF.type, URIRef("http://www.w3.org/ns/dcat#Dataset")))
g.add((URIRef(schemeURI + datasetStr), RDF.type, URIRef("http://publishmydata.com/pmdcat#Dataset")))
g.add((URIRef(schemeURI + datasetStr), URIRef("http://purl.org/dc/terms/modified"), Literal(datasetDateTime, datatype=XSD.dateTime)))
g.add((URIRef(schemeURI + datasetStr), URIRef("http://publishmydata.com/pmdcat#graph"), URIRef(schemeURI)))
g.add((URIRef(schemeURI + datasetStr), URIRef("http://purl.org/dc/terms/issued"), Literal(datasetDateTime, datatype=XSD.dateTime)))
g.add((URIRef(schemeURI + datasetStr), RDFS.label, Literal(mainLabel)))
g.add((URIRef(schemeURI + datasetStr), URIRef("http://publishmydata.com/pmdcat#datasetContents"), URIRef(schemeURI)))
g.add((URIRef(schemeURI + datasetStr), RDFS.comment, Literal(desc, lang="en")))
g.add((URIRef(schemeURI + datasetStr), URIRef("http://purl.org/dc/terms/title"), Literal(mainLabel)))


g.add((URIRef("http://gss-data.org.uk/catalog/vocabularies"), URIRef("http://www.w3.org/ns/dcat#record"), URIRef(schemeURI + catrecStr)))

catRecDateTime = "2020-12-23T11:58:58.120533"
g.add((URIRef(schemeURI + catrecStr), RDF.type, URIRef("http://www.w3.org/ns/dcat#CatalogRecord")))
g.add((URIRef(schemeURI + catrecStr), URIRef("http://purl.org/dc/terms/title"), Literal(mainLabel, lang="en")))
g.add((URIRef(schemeURI + catrecStr), URIRef("http://purl.org/dc/terms/issued"), Literal(catRecDateTime, datatype=XSD.dateTime)))
g.add((URIRef(schemeURI + catrecStr), URIRef("http://xmlns.com/foaf/0.1/primaryTopic"), URIRef(schemeURI + '/dataset')))
g.add((URIRef(schemeURI + catrecStr), URIRef("http://purl.org/dc/terms/modified"), Literal(catRecDateTime, datatype=XSD.dateTime)))
g.add((URIRef(schemeURI + catrecStr), URIRef("http://publishmydata.com/pmdcat#metadataGraph"), URIRef(schemeURI)))

for row in maindat.iterrows():
    if 'None' not in row[1]['Main']:
        conceptURI = URIRef(repoBase + "/" + con + "/" + main + "/" + str(pathify(row[1]['Main'])).upper())
        g.add((conceptURI, RDF.type, SKOS.Concept))
        g.add((conceptURI, RDFS.label, Literal(row[1]['Main Label'], lang="en")))
        g.add((conceptURI, SKOS.prefLabel, Literal(row[1]['Main Label'], lang="en")))
        g.add((conceptURI, SKOS.altLabel, Literal(row[1]['Main Label'] + " - " + row[1]['Main'], lang="en")))
        g.add((conceptURI, SKOS.inScheme, URIRef(schemeURI)))
        g.add((conceptURI, SKOS.notation, Literal(str(pathify(row[1]['Main'])).upper())))
        #### GEMOETRY
        if 'None' not in row[1]['Coordinates']:
            conceptGeom = URIRef(repoBase + "/" + con + "/" + main + "/" + str(pathify(row[1]['Main'])).upper() + '/geometry')
            g.add((conceptURI, hasGeom, conceptGeom))
            g.add((conceptGeom, RDF.type, geomURI))
            geomCoords = Literal(row[1]['Shape'].upper() + ' ' + str(row[1]['Coordinates']), datatype=URIRef("http://www.opengis.net/ont/geosparql#wktLiteral"))
            #g.add((conceptURI, hasGeom, geomCoords))
            g.add((conceptGeom, geomSpa, geomCoords))
           
        if 'None' not in row[1]['Narrower']:
            broaderURI = URIRef(repoBase + "/" + con + "/" + main + "/" + str(pathify(row[1]['Narrower']).upper()))
            g.add((conceptURI, SKOS.Narrower, broaderURI))
        try:
            if 'None' not in row[1]['Main Description']:
                g.add((conceptURI, RDFS.comment, Literal(row[1]['Main Description'], lang="en")))
        except:
            #print(row[1]['Main Description'])
            i = 0
#        #g.add((conceptURI, URIRef(sortPri), Literal(i)))
#        i = i + 1   
    if 'None' not in row[1]['Narrower']:
        conceptURI = URIRef(repoBase + "/" + con + "/" + main + "/" + str(pathify(row[1]['Narrower'])).upper())
        g.add((conceptURI, RDF.type, SKOS.Concept))
        g.add((conceptURI, RDFS.label, Literal(row[1]['Narrow Label'], lang="en")))
        g.add((conceptURI, SKOS.prefLabel, Literal(row[1]['Narrow Label'], lang="en")))
        g.add((conceptURI, SKOS.altLabel, Literal(row[1]['Narrow Label'] + " - " + row[1]['Narrower'], lang="en")))
        g.add((conceptURI, SKOS.inScheme, URIRef(schemeURI)))
        g.add((conceptURI, SKOS.notation, Literal(str(pathify(row[1]['Narrower'])).upper())))
        if 'None' not in row[1]['Main']:
            broaderURI = URIRef(repoBase + "/" + con + "/" + main + "/" + str(pathify(row[1]['Main']).upper()))
            g.add((conceptURI, SKOS.broader, broaderURI))
        try:
            if 'None' not in row[1]['Narrow Description']:
                g.add((conceptURI, RDFS.comment, Literal(row[1]['Narrow Description'], lang="en")))
        except:
            #print(row[1]['Narrow Description'])
            i = 0
        #g.add((conceptURI, URIRef(sortPri), Literal(i)))
        i = i + 1
        
schemeURI = URIRef(repoBase + "/" + con + '-' + sch + "/" + "sdmx-area-code-list")
for c in uniqueCodes:   
    conceptURI = URIRef(repoBase + "/" + con + "/" + "sdmx-area-code-list" + "/" + c)
    g.add((URIRef(schemeURI), SKOS.hasTopConcept, URIRef(conceptURI)))
  
#g.bind("cogs", schemeURI + "/")
g.bind("skos", SKOS)
g.bind("rdf", RDF)
g.bind("rdfs", RDFS)
g.bind("dc", URIRef("http://purl.org/dc/terms/"))
g.bind("prov", URIRef("http://www.w3.org/ns/prov#"))
g.bind("vocab", URIRef("http://gss-data.org.uk/catalog/"))
g.bind("pmdcat", URIRef("http://publishmydata.com/pmdcat#"))
g.bind("dcat", URIRef("http://www.w3.org/ns/dcat#"))
g.bind("foaf", URIRef("http://xmlns.com/foaf/0.1/"))
#print(g.serialize(format='n3').decode("utf-8"))

g.serialize(destination='cl-area.ttl', format='turtle')

In [245]:
# Need to get rid of the @prefixs in th output file, probably can do this within RDFLIB but i'm doing it this way!!
repl = {  
"vocab:vocabularies": "<http://gss-data.org.uk/catalog/vocabularies>",
"rdfs:label": "<http://www.w3.org/2000/01/rdf-schema#label>",
"rdfs:seeAlso": "<http://www.w3.org/2000/01/rdf-schema#seeAlso>",
"rdfs:comment": "<http://www.w3.org/2000/01/rdf-schema#comment>",
"skos:ConceptScheme": "<http://www.w3.org/2004/02/skos/core#ConceptScheme>",
"skos:Concept": "<http://www.w3.org/2004/02/skos/core#Concept>",
"skos:altLabel": "<http://www.w3.org/2004/02/skos/core#altLabel>",
"skos:inScheme": "<http://www.w3.org/2004/02/skos/core#inScheme>",
"skos:notation": "<http://www.w3.org/2004/02/skos/core#notation>",
"skos:prefLabel": "<http://www.w3.org/2004/02/skos/core#prefLabel>",
"skos:broader": "<http://www.w3.org/2004/02/skos/core#broader>",
"skos:Narrower": "<http://www.w3.org/2004/02/skos/core#narrower>",
"skos:hasTopConcept": "<http://www.w3.org/2004/02/skos/core#hasTopConcept>",    
"dcat:record": "<http://www.w3.org/ns/dcat#record>",
"dcat:CatalogRecord": "<http://www.w3.org/ns/dcat#CatalogRecord>",
"dcat:Dataset": "<http://www.w3.org/ns/dcat#dataset>",
"dc:issued": "<http://purl.org/dc/terms/issued>",
"dc:modified": "<http://purl.org/dc/terms/modified>",
"dc:title": "<http://purl.org/dc/terms/title>",
"xsd:dateTime": "<http://www.w3.org/2001/XMLSchema#dateTime>",
"foaf:primaryTopic": "<http://xmlns.com/foaf/0.1/primaryTopic>",
"pmdcat:metadataGraph": "<http://publishmydata.com/pmdcat#metadataGraph>",
"pmdcat:Dataset": "<http://publishmydata.com/pmdcat#Dataset>",
"pmdcat:datasetContents": "<http://publishmydata.com/pmdcat#datasetContents>",
"pmdcat:graph": "<http://publishmydata.com/pmdcat#graph>",
"pmdcat:ConceptScheme": "<http://publishmydata.com/pmdcat#ConceptScheme>",
"xsd:dateTime": "<http://www.w3.org/2001/XMLSchema#dateTime>",
"prov:hadDerivation": "<http://www.w3.org/ns/prov#hadDerivation>",
"ns1:hasGeometry": "<http://www.opengis.net/ont/geosparql#hasGeometry>",
"ns1:wktLiteral": "<http://www.opengis.net/ont/geosparql#wktLiteral>",
"ns1:asWKT": "<http://www.opengis.net/ont/geosparql#asWKT>",
"ns1:Geometry": "<http://www.opengis.net/ont/geosparql#Geometry>"
}

pref = """@prefix dc: <http://purl.org/dc/terms/> .
@prefix dcat: <http://www.w3.org/ns/dcat#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix ns1: <http://www.opengis.net/ont/geosparql#> .
@prefix pmdcat: <http://publishmydata.com/pmdcat#> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix vocab: <http://gss-data.org.uk/catalog/> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> ."""

In [246]:
f = open("cl-area.ttl", "r")
#f = open("period.ttl", "r")
txt = f.read()
f.close
for a in repl:
    txt = txt.replace(str(a),repl[a])

#txt = txt.replace(pref,'')
txt = txt.strip()
f = open("cl-area.ttl", "w")
#f = open("period.ttl", "w")
f.write(txt)
f.close()

In [103]:
now = datetime.datetime.now()
print(now.year, now.month, now.day, now.hour, now.minute, now.second)

2021 5 4 15 33 53


In [104]:

g = Graph()
sch = "<http://gss-data.org.uk/data/gss_data/trade/ons-quarterly-national-accounts#scheme/period>"
f = open("period.ttl", "w")
f.write("""PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dc: <http://purl.org/dc/terms/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX pmdcat: <http://publishmydata.com/pmdcat#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
       
<http://gss-data.org.uk/data/gss_data/trade/ons-quarterly-national-accounts#scheme/period> rdf:type pmdcat:ConceptScheme.

<http://gss-data.org.uk/data/gss_data/time/httpgss-dataorgukdatagss_datatradeons-quarterly-national-accountsschemeperiod/dataset> 
    rdf:type pmdcat:Dataset, dcat:Dataset;
    pmdcat:datasetContents <http://gss-data.org.uk/data/gss_data/trade/ons-quarterly-national-accounts#scheme/period>; \n
    dc:title "Period";
    rdfs:label "Period";
    dc:issued "2000-01-01T00:00:00Z"^^xsd:dateTime;
    dc:modified "2000-01-01T00:00:00Z"^^xsd:dateTime;
    pmdcat:graph <http://gss-data.org.uk/data/gss_data/trade/ons-quarterly-national-accounts#scheme/period>.

<http://gss-data.org.uk/data/gss_data/time/httpgss-dataorgukdatagss_datatradeons-quarterly-national-accountsschemeperiod/catalog-record> 
    rdf:type dcat:CatalogRecord;
    dc:issued "2021-04-15T12:51:09.472Z"^^xsd:dateTime;
    dc:modified "2021-04-15T12:51:09.472Z"^^xsd:dateTime;
    dc:title "Period Catalog Record";
    rdfs:label "Period Catalog Record";
    foaf:primaryTopic <http://gss-data.org.uk/data/gss_data/time/httpgss-dataorgukdatagss_datatradeons-quarterly-national-accountsschemeperiod/dataset>;
    pmdcat:metadataGraph <http://gss-data.org.uk/data/gss_data/trade/ons-quarterly-national-accounts#scheme/period>. \n""")

f.write(" <http://gss-data.org.uk/catalog/vocabularies> dcat:record <http://gss-data.org.uk/data/gss_data/time/httpgss-dataorgukdatagss_datatradeons-quarterly-national-accountsschemeperiod/catalog-record>. \n")

for x in range(1940, 2022):
    ystr = str(x)
    yuri = f"<http://reference.data.gov.uk/id/year/{ystr}>"
    f.write(f"{yuri} a skos:Concept ;\n")
    f.write(f"    skos:inScheme {sch} ;\n")
    f.write(f'''    rdfs:label "{ystr}" .\n''')
    f.write(f"{sch} skos:hasTopConcept {yuri} .\n")
    for y in range(1, 5):
        qstr = str(x) + "-Q" + str(y)
        quri = f"<http://reference.data.gov.uk/id/quarter/{qstr}>"
        f.write(f"{quri} a skos:Concept ;\n")
        f.write(f"    skos:inScheme {sch} ;\n")
        f.write(f'''   rdfs:label "{qstr}" .\n''')
        f.write(f"{sch} skos:hasTopConcept {quri} .\n")
        
f.close()   


SyntaxError: invalid syntax (<ipython-input-104-4a73bbac6d93>, line 5)

In [87]:
#coords = get_coordinates()
#coords.head(20)

,Main Label,ISO,Shape,Coordinates
0,Aruba,ABW,Polygon,"(((-69.99693762899992, 12.577582098000036), (-..."
0,Afghanistan,AFG,Polygon,"(((71.04980228700009, 38.40866445000009), (71...."
0,Angola,AGO,MultiPolygon,"((((11.73751945100014, -16.692577982999836), (..."
0,Anguilla,AIA,MultiPolygon,"((((-63.037668423999946, 18.21295807500003), (..."
0,Albania,ALB,Polygon,"(((19.747765747000074, 42.57890085900007), (19..."
0,Aland,ALA,MultiPolygon,"((((20.920176629000082, 59.90924713700012), (2..."
0,Andorra,AND,Polygon,"(((1.707006470000067, 42.5027814740001), (1.69..."
0,United Arab Emirates,ARE,MultiPolygon,"((((53.863047722, 24.23468659100014), (53.8886..."
0,Argentina,ARG,MultiPolygon,"((((-68.65412350199998, -54.88624439899996), (..."
0,Armenia,ARM,MultiPolygon,"((((45.54717289200002, 40.62155263300009), (45..."
